#### Camada Silver: Limpeza e Transformação

In [0]:
# Importar as bibliotecas necessárias
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Iniciar a SparkSession com configurações otimizadas
spark = SparkSession.builder \
    .appName("Transformação Data Silver") \
    .config("spark.sql.shuffle.partitions", "200")  \
    .config("spark.sql.files.maxPartitionBytes", "128MB") \
    .config("spark.sql.parquet.compression.codec", "snappy") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

bronze_path = '/FileStore/lhdw/bronze/vendas'
silver_path = '/FileStore/lhdw/silver/vendas'

In [0]:
#dbutils.fs.ls(bronze_path)

####Cria a lista com os arquivos que serão processados para a camada SILVER

In [0]:

list_files = spark.read.csv('/FileStore/lhdw/bronze/vendas/files_to_silver_process',header=True,inferSchema=True)\
            .select('filename').distinct().collect()
#print(list_files)

####Processamento da Camada SILVER

In [0]:
for row in list_files:
    #Importa os dados da camada Bronze para transformação na camada Silver
    df_bronze = spark.read.parquet(f'{bronze_path}/{row.filename}').distinct()
    
    #Transformação para a camada Silver
    df_silver = df_bronze.withColumn('Data',to_date('data','yyyy-MM-dd'))\
                    .withColumn('Email',lower(regexp_replace(split('EmailNome',':')[0],'[()]','')))\
                    .withColumn('Nome', expr("split(split(EmailNome,':')[1],', ')"))\
                    .withColumn('Nome',expr("concat(Nome[1],' ',Nome[0])"))\
                    .withColumn('Cidade',split('Cidade',',')[0])\
                    .withColumn('PrecoUnitario',format_number(col('PrecoUnitario'),2).cast('double'))\
                    .withColumn('CustoUnitario',format_number(col('CustoUnitario'),2).cast('double'))\
                    .withColumn('TotalVendas',format_number(col('PrecoUnitario')*col('Unidades'),2).cast('double'))\
                    .drop('EmailNome')\
                    .drop('IDCampanha')

    #Gravar transformações na Camada Silver
    df_silver.withColumn('Ano',year('Data'))\
         .withColumn('Mes',month('Data'))\
         .write.partitionBy('Ano','Mes')\
         .format('parquet')\
         .mode('append')\
         .save(silver_path)

In [0]:
spark.read.parquet(silver_path).select('filename').groupBy('filename').count().display()

####Deleta arquivo 'files_to_silver_process', pois todos os arquivos foram processados.

In [0]:
dbutils.fs.rm('/FileStore/lhdw/bronze/vendas/files_to_silver_process',True)

### Limpeza de Memória

In [0]:
del df_bronze
del df_silver